In [1]:
from espnet.utils.io_utils import LoadInputsAndTargets
import json
from kaldialign import align, edit_distance
from espnet.asr.pytorch_backend.asr_init import load_trained_model
from espnet.bin.asr_recog import get_parser
from espnet.utils.deterministic_utils import set_deterministic_pytorch
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm.notebook import tqdm

/home/neo/espnet/tools/venv/lib/python3.8/site-packages/chainer/backends/cuda.py:142: UserWarning: cuDNN is not enabled.
Please reinstall CuPy after you install cudnn
(see https://docs-cupy.chainer.org/en/stable/install.html#install-cudnn).
  warnings.warn(


In [2]:
model, train_args = load_trained_model('/home/neo/espnet/egs/timit/asr1/exp/train_nodev_pytorch_test/results/model.acc.best', training=False)

In [3]:
parser = get_parser()

In [4]:
args = parser.parse_args(f'--config  /home/neo/espnet/egs/timit/asr1/conf/decode.yaml \
                          --ngpu 1 --backend pytorch --batchsize 1 --result-label results.json \
                          --model /home/neo/espnet/egs/timit/asr1/exp/train_nodev_pytorch_test/results/model.acc.best \
                          --api v1')
set_deterministic_pytorch(args)
model.recog_args = args

In [5]:
args.beam_size = 10
args.lm_weight = 0.0
args.ctc_weight = 0.4
args.nbest = 1
args.verbose = 4
args.debugmode = 2
model.recog_args = args

In [16]:
with open('/home/neo/espnet/egs/timit/asr1/dump/test/deltafalse/data.json', "rb") as f:
    js = json.load(f)["utts"]

In [17]:
load_inputs_and_targets = LoadInputsAndTargets(
        mode="asr",
        load_output=True,
        sort_in_input_length=False,
        preprocess_conf=None,
        preprocess_args={"train": True},
)

In [18]:
def get_cer(ref, hypo):
    ref = [str(ele) for ele in ref[0]]
    hyp = [str(ele) for ele in hypo[0]['yseq'][1:-1]]
    alignment = align(ref, hyp, '*')
    edit_stats = edit_distance(ref, hyp)
    cer = (edit_stats['ins'] + edit_stats['del'] + edit_stats['sub'])/len(ref)
    return cer, alignment

In [19]:
def get_logits_history(hypo):
    return torch.stack(hypo[0]['logits_history'][1:-1]).squeeze(axis=1).detach().numpy()
def get_att_c_history(hypo):
    return torch.stack(hypo[0]['att_c_history'][1:-1]).squeeze(axis=1).detach().numpy()
def get_att_w_history(hypo):
    return torch.stack(hypo[0]['att_w_history'][-1][1:-1]).squeeze(axis=1).detach().numpy()
def get_decoder_states_history(hypo):
    return torch.stack([ele[0] for ele in hypo[0]['decoder_states_history'][1:-1]]).squeeze(axis=1).detach().numpy()

In [20]:
for name in tqdm(js.keys()):
    batch = [(name, js[name])]
    feats, labels = load_inputs_and_targets(batch)
    hypo = model.recognize(feats[0], model.recog_args, train_args.char_list)
    stats = {}
    stats['cer'] = get_cer(labels, hypo)
    stats['logits_history'] = get_logits_history(hypo)
    stats['att_w_history'] = get_att_w_history(hypo)
    stats['att_c_history'] = get_att_c_history(hypo)
    stats['decoder_states_history'] = get_decoder_states_history(hypo)
    np.save(f'/home/neo/Desktop/projects/perf_monitor/test/{name}.npy', stats)

/home/neo/espnet/espnet/nets/ctc_prefix_score.py:276: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(
